<a href="https://colab.research.google.com/github/aymericvie/Covid19_coevolution/blob/main/Covid_coevolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [200]:
# imports
import random
import numpy as np
random.seed(9)

# Parameters
virus_size = 5
initial_virus_population = 1
baseR = round(random.uniform(2,3),2)
#print(baseR)
mut = np.sort(np.random.normal(0,1,size=virus_size))
#print(mut)

policy_size = 6
initial_policy_population = 4

initial_infected = 10

mutation_rate_virus = 0.5

# Parameters for non pharmaceutical interventions (NPIs) from Haug et al (2020)
npi = [round(random.uniform(-0.36,-0.18),2), round(random.uniform(-0.24,-0.16),2), round(random.uniform(-0.24,-0.08),2), round(random.uniform(-0.18,-0.08),2), round(random.uniform(-0.18,-0.04),2), round(random.uniform(-0.18,0),2)]
#print(npi)

# Initial sampling for viruses
population_viruses = np.zeros((initial_virus_population,virus_size))
print(population_viruses)

# Initial sampling for policies
population_policies = np.zeros((initial_policy_population,policy_size))
#print(population_policies)

infected = initial_infected
population_virus_size = initial_virus_population
population_policy_size = initial_policy_population

###

population_policies[0,0] = 1
population_policies[0,1] = 1
population_policies[0,2] = 1
population_policies[1,1] = 1
population_policies[2,2] = 1
population_policies[3,3] = 1

print("initial pop polici")
print(population_policies)

# population_viruses[0,0] = 1
# population_viruses[1,1] = 1
# population_viruses[2,2] = 1
# population_viruses[3,3] = 1
# population_viruses[4,4] = 1

# determine efficacy of population viruses and policies
virusR = np.zeros((population_virus_size,1))
i = 0
while i < population_virus_size:
  virusR[i,0] = baseR + np.sum(population_viruses, axis = 1)[i]
  i += 1
print(virusR)

policyR = np.zeros((population_policy_size,1))
i = 0
while i < population_policy_size:
  policyR[i,0] = np.dot(population_policies[i,],npi)
  i += 1
#print(policyR)

# evaluate coevolution fitness evaluation for policies, with tournament(3) selection
policy_fitness = np.zeros((population_policy_size,1))

i = 0
while i < population_policy_size:
  a = random.randint(0,population_virus_size-1)
  b = random.randint(0,population_virus_size-1)
  c = random.randint(0,population_virus_size-1)
  policy_fitness[i,0] = (1/3)*(virusR[a,0] + virusR[b,0] + virusR[c,0]) + policyR[i,0]
  i += 1
#print(policy_fitness)
adjusted_policy_fitness = 1/(1+policy_fitness)
#print(adjusted_policy_fitness)
policy_selection = adjusted_policy_fitness / np.sum(adjusted_policy_fitness)
#print(policy_selection)

cumulative_policy_selection = np.copy(policy_selection)
i = 1
while i < len(cumulative_policy_selection):
  cumulative_policy_selection[i,0] += cumulative_policy_selection[i-1,0]
  i += 1
#print(cumulative_policy_selection)

i = 0
while i < population_policy_size:
  a = random.uniform(0,1)
  #print(a)
  b = 0
  while a > cumulative_policy_selection[b]:
      b += 1
  #print(b)

  a = random.uniform(0,1)
  #print(a)
  c = 0
  while a > cumulative_policy_selection[c]:
      c += 1
  #print(c)

  #print(population_policies[b,])
  #print(population_policies[c,])

# Do crossover between elements b and c
  cpoint = random.randint(0,policy_size)
  #print(cpoint)
  offspring1 = np.zeros((1, policy_size))
  offspring2 = np.zeros((1, policy_size))

  j = 0
  while j < cpoint:
    offspring1[0,j] = population_policies[b,j]
    offspring2[0,j] = population_policies[c,j]
    j += 1
  while j < policy_size:
    offspring1[0,j] = population_policies[c,j]
    offspring2[0,j] = population_policies[b,j]
    j += 1
  #print(offspring1)
  #print(offspring2)

  ####
  # IMPLEMENT MUTATIONS
  ####

  population_policies = np.vstack([population_policies,offspring1])
  population_policies = np.vstack([population_policies,offspring2])
  i += 2

print("new pop polici")
population_policies = population_policies[population_policy_size:, :]
print(population_policies)


# operate virus transmission and mutation

print(population_viruses)
print(virusR)
i = 0
while i < population_virus_size:
  j = 0
  while j < int(virusR[i,0]):
    candidate_virus = np.copy(population_viruses[i,])
    k = 0
    while k < virus_size:
      n = random.uniform(0,1)
      if n <= mutation_rate_virus:
        print("mutation at" + str(k))
        if candidate_virus[k] == 0:
          candidate_virus[k] = 1
        if candidate_virus[k] == 1:
          candidate_virus[k] = 0
      k += 1
    print(candidate_virus)
    population_viruses = np.vstack([population_viruses,candidate_virus])
    print("j" + str(j))
    j += 1
  i += 1
print(population_viruses.shape[0])
print("new pop virus")
population_viruses = population_viruses[population_virus_size:, :]
print(population_viruses)
population_virus_size = population_viruses.shape[0]
print(population_virus_size)



[[0. 0. 0. 0. 0.]]
initial pop polici
[[1. 1. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]
[[2.46]]
new pop polici
[[1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]]
[[2.46]]
mutation at1
mutation at2
[0. 0. 0. 0. 0.]
j0
mutation at0
mutation at1
mutation at2
mutation at3
mutation at4
[0. 0. 0. 0. 0.]
j1
3
new pop virus
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
2


In [198]:
candidate_virus[0] = 1

In [199]:
print(candidate_virus)

[1. 0. 0. 0. 0.]
